In [1]:
import pandas as pd
pd.options.display.max_rows = 350
from sklearn import preprocessing

STATES = False

In [2]:
master = pd.read_csv('../county-data.csv', index_col=0)

data = master[master.CM_age_adj_rate_opiates.isnull() == False]
deaths = master[master.CM_age_adj_rate_opiates.isnull() == False].CM_age_adj_rate_opiates

# Drop the target column as well as all the other columns that would seem to be data leaks
data = data.drop(['DM_deaths_drug', 'DM_crude_rate_drug', 'DM_age_adj_rate_drug',
                  'CM_deaths_opiates', 'CM_crude_rate_opiates', 'CM_age_adj_rate_opiates'], axis=1)

data = pd.get_dummies(data, columns=['census_div', 'topo_code', 'RUCC_1974', 'RUCC_1983', 'RUCC_1993', 'RUCC_2003',
                                     'RUCC_2013', 'UIC_1993', 'UIC_2003', 'UIC_2013'], drop_first=True)

if STATES:
    data = pd.get_dummies(data, columns=['state'], drop_first=True)
    data.drop(['county'], axis=1, inplace=True)
else:
    data.drop(['county', 'state'], axis=1, inplace=True)



for col in data:
    data[col] = (data[col]-data[col].mean())/(data[col].max()-data[col].min())

data.fillna(0, inplace=True)

In [3]:
len(deaths)

1755

In [4]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoLars, LassoLarsCV, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, deaths, random_state=1337)

In [6]:
#model = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=1337)
#model = Ridge(random_state=1337)

#model = Lasso(random_state=1337, max_iter=50)
#model = ElasticNet(random_state=1337, max_iter=100000, alpha=0.1)
model = LassoLarsCV(n_jobs=-1, cv=100, max_iter=20000, normalize=False)

cross_val_score(model, data, deaths, n_jobs=-1, cv=10)

In [7]:
model.fit(X_train, y_train)

/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 255 iterations, i.e. alpha=2.396e-03, with an active set of 211 regressors, and the smallest cholesky pivot element being 3.650e-08
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:381: RuntimeWarning: overflow encountered in true_divide
  g2 = arrayfuncs.min_pos((C + Cov) / (AA + corr_eq_dir + tiny))
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 239 iterations, i.e. alpha=2.717e-03, with an active set of 199 regressors, and the smallest cholesky pivot element being 2.980e-08
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWa

/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 271 iterations, i.e. alpha=2.129e-03, with an active set of 223 regressors, and the smallest cholesky pivot element being 5.162e-08
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 242 iterations, i.e. alpha=2.650e-03, with an active set of 198 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 272 iterations, i.e. alpha=2.129e-03, with an active set of 224 regressors, and the smallest cholesky pivot element being 4.215e-08
  Convergen

/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 352 iterations, alpha=1.421e-03, previous alpha=1.421e-03, with an active set of 287 regressors.
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 382 iterations, alpha=1.271e-03, previous alpha=1.271e-03, with an active set of 309 regressors.
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 350 iterations, i.e. alpha=1.448e-03, with an active set of 286 regressors, and the smallest cholesky pivot e

/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 298 iterations, i.e. alpha=1.833e-03, with an active set of 240 regressors, and the smallest cholesky pivot element being 2.980e-08
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 262 iterations, alpha=2.312e-03, previous alpha=2.312e-03, with an active set of 219 regressors.
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 313 iterations, i.e. alpha=1.574e-03, with an active set of 261 regressors, and the smallest cholesky pivot element being 4.21

/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 279 iterations, i.e. alpha=1.839e-03, with an active set of 235 regressors, and the smallest cholesky pivot element being 2.980e-08
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 290 iterations, i.e. alpha=1.721e-03, with an active set of 246 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 291 iterations, alpha=1.721e-03, previous alpha=1.721e-03, with an active set of 246 regres

...

/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 571 iterations, i.e. alpha=5.672e-04, with an active set of 425 regressors, and the smallest cholesky pivot element being 7.300e-08
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 687 iterations, i.e. alpha=3.532e-04, with an active set of 499 regressors, and the smallest cholesky pivot element being 5.162e-08
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 697 iterations, alpha=8.229e-04, previous alpha=3.443e-04, with an active set of 504 regres

/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 327 iterations, alpha=1.602e-03, previous alpha=1.602e-03, with an active set of 262 regressors.
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 434 iterations, alpha=1.022e-03, previous alpha=1.022e-03, with an active set of 345 regressors.
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 309 iterations, i.e. alpha=1.766e-03, with an active set of 249 regressors, and the smallest cholesky pivot e

/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 384 iterations, i.e. alpha=1.238e-03, with an active set of 316 regressors, and the smallest cholesky pivot element being 2.107e-08
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 385 iterations, alpha=1.238e-03, previous alpha=1.238e-03, with an active set of 316 regressors.
  ConvergenceWarning)
/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 379 iterations, i.e. alpha=1.218e-03, with an active set of 315 regressors, and the smallest cholesky pivot element being 2.22

/home/chrislit/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 859 iterations, alpha=2.000e-04, previous alpha=2.000e-04, with an active set of 574 regressors.
  ConvergenceWarning)


LassoLarsCV(copy_X=True, cv=100, eps=2.2204460492503131e-16,
      fit_intercept=True, max_iter=20000, max_n_alphas=1000, n_jobs=-1,
      normalize=False, positive=False, precompute='auto', verbose=False)

In [8]:
pred = model.predict(X_test)

In [9]:
r2_score(y_test, pred)

0.25886873080003681

In [10]:
mean_squared_error(y_test, pred)

7.5538191588132646

In [11]:
mean_absolute_error(y_test, pred)

1.8001462691165027

In [12]:
coeffs = zip(data.columns, model.coef_)
# Filter out 0ed coefficients
coeffs = [x for x in coeffs if x[1] != 0]

In [13]:
# Copied from https://pandas.pydata.org/pandas-docs/stable/style.html
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

df = pd.DataFrame(sorted(coeffs, key=lambda x: abs(x[1]), reverse=True), columns=['variable', 'coefficient'])
df.set_index('variable', inplace=True)
df.style.applymap(color_negative_red)

,coefficient
variable,
UCRP_cocaine_sale,15.1629
DM_crude_rate_all,6.21175
RCMS_congregations_cgma,4.41297
CM_age_adj_rate_all,3.51237
RCMS_pct_bprt,-2.9718
RCMS_pct_awmc,2.6672
RCMS_pct_grk,2.61671
partd_opioid_rate_2014,2.3786
jul_temp_mean,-2.26719
